In [1]:
# importing all necessary libs
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objs as go
import dash
import os
import dash_core_components as dcc
import dash_html_components as html
from plotly.subplots import make_subplots

import warnings
warnings.filterwarnings("ignore")

curr_dir = os.getcwd()
parent_dir = os.path.dirname(curr_dir)


In [2]:
# Data reading
df_map = pd.read_excel('../data/aggregated_map_data.xlsx')
df_attacks = pd.read_excel('../data/aggregated_attacktype_data.xlsx')
df_yearly = pd.read_excel('../data/aggregated_yearly_data.xlsx')

In [7]:
df_yearly[df_yearly["iyear"].isin([2017,2016,2015,2014,2013])]

,Unnamed: 0,iyear,country,country_txt,id,isOrganized,nkill,nwound,total_attacks,total_kill_injured
0,0,2017,4,Afghanistan,AFG,organized,4235,4292,815,8527
12,12,2017,4,Afghanistan,AFG,unorganized,422,721,356,1143
93,93,2017,5,Albania,ALB,organized,0,0,0,0
94,94,2017,5,Albania,ALB,unorganized,0,0,1,0
123,123,2017,6,Algeria,DZA,unorganized,1,7,7,8
...,...,...,...,...,...,...,...,...,...,...
7507,7507,2017,1001,Republic of Serbia,SRB,organized,0,0,0,0
7539,7539,2017,1003,Kosovo,CS-KM,organized,0,1,1,1
7555,7555,2017,1003,Kosovo,CS-KM,unorganized,0,3,7,3
7557,7557,2017,1004,South Sudan,SSD,organized,244,123,23,367


In [3]:
with open(os.path.abspath('../data/data_description.txt')) as f:
    desc = f.readlines()
f.close()

In [16]:
# World map
def update_map():
    
    df_map = df_yearly[df_yearly["iyear"].isin([2017,2016,2015,2014,2013])]

    df_map['hover_text'] = 'Country: ' + df_map["country_txt"].astype(str) + "<br>" + 'Terrorism Index: ' + df_map['total_attacks'].round(decimals= 2).astype(str) + '<br>'+'# of Killed and Wounded People: ' +  df_map['total_kill_injured'].astype(str)+ '<br>'+'# of Killed People: '+  df_map['nkill'].astype(str)+ '<br>'+ '# of Wounded People: '+  df_map['nwound'].astype(str)
    
    trace = px.scatter_geo(df_map, locations=df_map['id'],
                          size=df_map['total_attacks'],
                          text=df_map['hover_text'], 
                          #hover='text' ,
                          #colorscale="Reds",
                          #reversescale = True,
                          #marker={'line': {'color': 'rgb(180,180,180)','width': 0.5}},
                          #colorbar={"thickness": 20,"len": 0.7, "x": 0.9, "y": 0.7,
                          #         'title': {"text": 'Terrorism Index', "side": "bottom"},
                                # }
                         )   
    return {"data": [trace],
            "layout": go.Layout(height=800, width= 1200 ,
                                geo={'showframe': False,
                                     'showcoastlines': False,
                                     'projection': {'type': "miller"}})}



external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
app.title = 'Dashboard'


app.layout = html.Div([
    html.Div([html.H1("Global Terrorism Dashboard")],
                 style={'textAlign': "center", "padding-bottom": "30"}, 
                 className="six-column"),
    html.Div([html.P(dcc.Markdown(desc))],
                 style={'textAlign': "left", "padding-bottom": "30"}, 
                 className="six-column"),
    html.Div([html.H3('Terrorism Index Map',
                 style={'textAlign': "center", "padding-bottom": "30"}),
              html.P(dcc.Markdown('''Terrorism Index map shows the level of safety based on the number of terrorist
                           attacks since 1970. Map reflects the current level of safety based on 
                           terrorism index that gives more weight to the more recent years.'''),
                 style={'textAlign': "left", "padding-bottom": "30"}, 
                 className="six-column"),
              dcc.Graph(id="world-map", 
                       style = {'textAlign': "center"}, 
                       figure = update_map(), className="twelve columns"), 
              html.A('Source: Global Terrorism Database(GTD)',href = 'https://www.start.umd.edu/gtd/', 
                      style={'fontStyle': "italic", 'color': '#273746'})]), 
    html.Div([html.H3('Countries with the highest number of known and unknown affiliated attacks',
            style={'textAlign': "center", "padding-bottom": "30"}),
            dcc.Graph(id="horizontal-bar")], 
            className="twelve columns"),
    html.H3(id = 'country-name',  children=["init"] , 
            style={'textAlign': "center", "padding-bottom": "30"} ),
    html.Div(dcc.Graph(id="incidents"),  id = 'incident-box', 
             style = {'visibility': 'hidden'}, className="five columns"),
    html.Div(dcc.Graph(id="fatalities"),  id = 'fatalities-box', 
             style = {'visibility': 'hidden'}, className="five columns"),
    
])



##### Figure Updates with CallBacks ##### 
@app.callback(
    dash.dependencies.Output('country-name', 'children'),
    [dash.dependencies.Input('world-map', 'clickData')])
def update_title(clickData):
    text = ''
    if clickData is not None:
        selected_country = clickData['points'][0]['location']
        text = str(selected_country)  
    return text

@app.callback(
    dash.dependencies.Output('horizontal-bar', 'figure'),
    [dash.dependencies.Input('world-map', 'clickData')])
def update_horizontal_bar(clickData):

    fig = go.Figure()
    years = df_attacks['iyear'].sort_values().unique()

    for step in years:
        countries = df_attacks[df_attacks['iyear']==step]
        fig.add_trace(go.Bar(
            visible=False,
            y=countries['country_txt'],
            x=countries['organized'],
            name='Known',
            orientation='h',
            marker=dict(
                color='rgba(246, 78, 139, 0.6)',
                line=dict(color='rgba(246, 78, 139, 1.0)', width=3)
            )
        ))
        fig.add_trace(go.Bar(
            visible=False,
            y=countries['country_txt'],
            x=countries['unorganized'],
            name='Unknown',
            orientation='h',
            marker=dict(
                color='#5885AF',
                opacity = 0.8,
                line=dict(color='rgba(58, 71, 80, 1.0)', width=3)
            )
        ))

    fig.data[-1].visible = True
    fig.data[-2].visible = True

    steps = []
    for i in range(len(years)):
        step = dict(
            method="update",
            args=[{"visible": [False] * len(fig.data)},],  # layout attribute
        )
        step["args"][0]["visible"][i * 2] = True  # Toggle i'th trace to "visible"
        if i * 2 < len(fig.data) - 1:
            step["args"][0]["visible"][i * 2 + 1] = True  # Toggle i'th trace to "visible"
        steps.append(step)

    sliders = [dict(
        active=48,
        currentvalue={"prefix": "Selected Year: "},
        pad={"t": 50},
        steps=steps
    )]

    fig.update_layout(
        sliders=sliders,
        barmode='stack'
    )

    fig['layout']['sliders'][0]['currentvalue']['prefix']='Year: '
    for i, date in enumerate(years, start = 0):
        fig['layout']['sliders'][0]['steps'][i]['label']=str(date)
        
#     fig.update_layout(margin=dict(t=120))
    fig['layout']['sliders'][0]['pad']=dict(t= -360,)
    
    return fig
        



@app.callback(
    [dash.dependencies.Output('incidents', 'figure'), dash.dependencies.Output('incident-box', 'style')],
    [dash.dependencies.Input('world-map', 'clickData')])
def update_incidents(clickData):
    df_agg = ''
    if clickData is not None:
        selected_country =clickData['points'][0]['location']
        df_agg = df_yearly[df_yearly['id'] == selected_country].groupby(by=['iyear','country',
                                                                            'country_txt','id', 
                                                                            'isOrganized']).agg({
                                                                                        'total_attacks':'sum'}).reset_index()
    fig = px.line(df_agg, x="iyear", y="total_attacks", color='isOrganized',
                  labels={
                     "iyear": "<b>Year</b>",
                     "total_attacks": "<b>Number of Incidents</b>",
                     "isOrganized": " "
                 }, title="<b>Number of Incidents by Years</b>")
    
    return fig,  {'visibility':'visible'}

@app.callback(
    [dash.dependencies.Output('fatalities', 'figure'), dash.dependencies.Output('fatalities-box', 'style')],
    [dash.dependencies.Input('world-map', 'clickData')])
def update_fatalities(clickData):
    df_agg = ''
    
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    if clickData is not None:
        selected_country =clickData['points'][0]['location']
        df_agg = df_yearly[df_yearly['id'] == selected_country].groupby(by=['iyear','country',
                                                                        'country_txt','id']).agg({
                                                                                        'nkill':np.sum,
                                                                                        'nwound':np.sum  }).reset_index()
    # Add traces
    fig.add_trace(
        go.Line(x=df_agg['iyear'], y=df_agg['nwound'], name="Injuries", line=dict(color="#D4AC0D")),
        secondary_y=False,
    )

    fig.add_trace(
        go.Line(x=df_agg['iyear'], y=df_agg['nkill'], name="Fatalities", line=dict(color="#943126")),
        secondary_y=True,
    )     
    
    # Add figure title
    fig.update_layout(
        title_text="<b>Number of Injuries and Fatalities by Years</b>"
    )

    # Set x-axis title
    fig.update_xaxes(title_text="<b>Year</b>")

    # Set y-axes titles
    fig.update_yaxes(title_text="<b>Injuries</b>", secondary_y=False)
    fig.update_yaxes(title_text="<b>Fatalities</b> ", secondary_y=True)

    
    return fig, {'visibility':'visible'}
  
app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
